# Randomly assign mutations to samples in each cancer type

Based on the combination of risk factors, there are 27 possible samples per cancer type. For each of these, we will create 10 different random samples, each containing a fixed set of drivers but random set of passengers. 

We will export two tables (one for skin and another for lung cancers), containing 270 rows each. 

Only FDA approved therapies as annotated in CGI website (August 2024) are used. 

In [8]:
import pandas as pd
import numpy as np

#### Output files 

In [9]:
skin_output_f = 'skin_db.txt'
lung_output_f = 'lung_db.txt'

#### Samples

In [10]:
# Age
var_items1 = ['1a', '1b', '1c']
# Solar exposure / smoking
var_items2 = ['2a', '2b', '2c']
# Solar protection / passive smoking
var_items3 = ['3a', '3b', '3c']
# Sample number
var_items4 = [f'4{i}' for i in 'abcdefghij']
var_items4

['4a', '4b', '4c', '4d', '4e', '4f', '4g', '4h', '4i', '4j']

#### Passenger mutations (common between cancer types)

In [11]:
mutations_f = '/home/claudia/Claudia/outreach/cancerdetective/mutations_db.tsv'
mutations_df = pd.read_csv(mutations_f, sep='\t', header=0)
mutations_df.head(2)

,cancer_type,mutation_id,gene,description,aa_change,dna_change,driver_passenger,og_tsg,targeted_therapy,targeted_therapy_approved
0,lung,FBXW7_R505P,FBXW7,F-box and WD repeat domain containing 7,R505P,chr4:152326136_C>G,driver,tsg,None,False
1,lung,FAM135B_G186E,FAM135B,Family with sequence similarity 135 member B,G186E,chr8:138243054_C>T,driver,og,None,False


In [12]:
passengers_df = mutations_df.loc[mutations_df['driver_passenger'] == 'passenger']
print(len(passengers_df))
passengers_df.head(2)

193


,cancer_type,mutation_id,gene,description,aa_change,dna_change,driver_passenger,og_tsg,targeted_therapy,targeted_therapy_approved
528,lung,OR8B2_A119E,OR8B2,Olfactory receptor family 8 subfamily B member 2,A119E,chr11:124382988_G>T,passenger,unknown,None,False
529,lung,OR51A4_R124G,OR51A4,Olfactory receptor family 51 subfamily A member 4,R124G,chr11:4946731_T>C,passenger,unknown,None,False


In [13]:
passenger_list = passengers_df['mutation_id'].tolist()
len(passenger_list)

193

## Skin melanomas

#### Drivers

Genes: BRAF, NRAS, HRAS, KRAS, CDKN2A, NF1, TP53

Driver mutations are assigned based on TGCA-SKCM frequencies. This is a simplified version: 
- 50% of melanomas are BRAF mutant. 
- 30% of melanomas are Ras mutant: NRAS (most frequently), HRAS or KRAS. These three are mutually exclusive
- 10-15% of melanomas are CDKN2A, or NF1, or TP53 mutated
- BRAF mutations can occur alone or together with CDKN2A, NF1, TP53 
- Ras mutations can occur alone or together with CDKN2A, NF1, TP53

In [38]:
drivers_skin_dict = {
    'a': ['BRAF_V600E'], 
    'b': ['BRAF_V600K'], 
    'c': ['BRAF_V600E', 'TP53_R248G', 'NF1_R1276P'], 
    'd': ['BRAF_V600E', 'CDKN2A_P81L'], 
    'e': ['BRAF_V600E', 'NF1_R1276P'], 
    'f': ['NRAS_Q61H'], 
    'g': ['NRAS_Q61H', 'CDKN2A_P81L'], 
    'h': ['NRAS_Q61K', 'TP53_Y220C', 'PTEN_D162G'], 
    'i': ['KRAS_G12D', 'CDKN2A_P81L'], 
    'j': ['HRAS_G12V', 'TP53_L145R'],
}

In [39]:
therapies_skin = {
    'BRAF_V600E': 'Vemurafenib', 
    'BRAF_V600K': 'Vemurafenib + Cobimetinib', 
}

In [40]:
lines = []
ctype = 'a'
variable_mutation_values = {'a': 1, 'b':2, 'c':3}
for v1 in var_items1: 
    for v2 in var_items2: 
        for v3 in var_items3: 
            # Sample type 
            sample_type = '_'.join([v1, v2, v3])
            # Number of mutations to be assigned to the sample
            exp_mut_count = sum([variable_mutation_values[i[1]] for i in [v1, v2, v3]])
            
            # Iterate over the 10 random samples in each possible sample type (e.g., 1a_2a_3a)
            for v4 in var_items4: 
                # Get drivers
                sample_drivers = drivers_skin_dict[v4[1]]
                # Get random passengers
                passengers_n = exp_mut_count - len(sample_drivers)
                # Add 1 passenger if none
                passengers_n = 1 if passengers_n == 0 else passengers_n
                sample_passengers = list(np.random.choice(passenger_list, size=passengers_n, replace=False))
                # Get treatments
                treatments = list(set([(driver, therapies_skin[driver]) for driver in sample_drivers if driver in therapies_skin.keys()]))
                if treatments: 
                    treatments = ':'.join(treatments[0])
                else: 
                    treatments = ''
                lines.append(pd.DataFrame([[
                    ctype,
                    sample_type, 
                    sample_type + '_' + v4,
                    exp_mut_count,
                    ';'.join(sorted(sample_drivers)), 
                    treatments,
                    ';'.join(sorted(sample_passengers)), 
                    ';'.join(sorted(sample_drivers+sample_passengers))                    
                ]]))
melanomas_results = pd.concat(lines)
melanomas_results.columns = [
    'cancer_type',
    'sample_type', 
    'sample',
    'number_mutations', 
    'drivers', 
    'treatments',
    'passengers', 
    'total'
]

In [41]:
melanomas_results.head(15)

,cancer_type,sample_type,sample,number_mutations,drivers,treatments,passengers,total
0,a,1a_2a_3a,1a_2a_3a_4a,3,BRAF_V600E,BRAF_V600E:Vemurafenib,APOC3_L7V;OR51L1_P20S,APOC3_L7V;BRAF_V600E;OR51L1_P20S
0,a,1a_2a_3a,1a_2a_3a_4b,3,BRAF_V600K,BRAF_V600K:Vemurafenib + Cobimetinib,OR13H1_L36H;OR8D2_M145I,BRAF_V600K;OR13H1_L36H;OR8D2_M145I
0,a,1a_2a_3a,1a_2a_3a_4c,3,BRAF_V600E;NF1_R1276P;TP53_R248G,BRAF_V600E:Vemurafenib,GAST_D41N,BRAF_V600E;GAST_D41N;NF1_R1276P;TP53_R248G
0,a,1a_2a_3a,1a_2a_3a_4d,3,BRAF_V600E;CDKN2A_P81L,BRAF_V600E:Vemurafenib,SPZ1_E62K,BRAF_V600E;CDKN2A_P81L;SPZ1_E62K
0,a,1a_2a_3a,1a_2a_3a_4e,3,BRAF_V600E;NF1_R1276P,BRAF_V600E:Vemurafenib,OR5J2_S67L,BRAF_V600E;NF1_R1276P;OR5J2_S67L
0,a,1a_2a_3a,1a_2a_3a_4f,3,NRAS_Q61H,,IGFL1_D49N;OR2W1_M25I,IGFL1_D49N;NRAS_Q61H;OR2W1_M25I
0,a,1a_2a_3a,1a_2a_3a_4g,3,CDKN2A_P81L;NRAS_Q61H,,OR1L8_P287S,CDKN2A_P81L;NRAS_Q61H;OR1L8_P287S
0,a,1a_2a_3a,1a_2a_3a_4h,3,NRAS_Q61K;PTEN_D162G;TP53_Y220C,,C9_G75E,C9_G75E;NRAS_Q61K;PTEN_D162G;TP53_Y220C
0,a,1a_2a_3a,1a_2a_3a_4i,3,CDKN2A_P81L;KRAS_G12D,,OR52D1_E302K,CDKN2A_P81L;KRAS_G12D;OR52D1_E302K
0,a,1a_2a_3a,1a_2a_3a_4j,3,HRAS_G12V;TP53_L145R,,OR4D6_S67L,HRAS_G12V;OR4D6_S67L;TP53_L145R


In [42]:
# Check there are no repeated samples
len(melanomas_results), len(melanomas_results['total'].unique())

(270, 270)

In [43]:
# Export
melanomas_results.to_csv(skin_output_f, sep='\t', header=True, index=False)

## Lung cancers

#### Drivers

Genes: TP53, EGFR, KRAS, NFE2L2, KEAP1, CDKN2A, PTEN, CDH10, KMT2D, KMT2C

Driver mutations are assigned based on TGCA-LUSC and TCGA-LUAD frequencies. This is a simplified version: 
- 60-80% of lung cancers are TP53 mutant
- NFE2L2 and KEAP1 are mutually exclusive

In [44]:
drivers_lung_dict = {
    'a': ['TP53_R248G', 'KRAS_G12C', 'NFE2L2_G31R'], 
    'b': ['TP53_R273H', 'EGFR_L858R'], 
    'c': ['TP53_Y220C', 'EGFR_L861Q'], 
    'd': ['TP53_R248G', 'NFE2L2_G31R'], 
    'e': ['TP53_R273H', 'KEAP1_V155F'], 
    'f': ['EGFR_L861Q'], 
    'g': ['TP53_L145R', 'CDKN2A_P81L'], 
    'h': ['TP53_P151S', 'EGFR_L858R', 'PTEN_D162G'], 
    'i': ['KEAP1_C23Y', 'EGFR_L861Q'], 
    'j': ['EGFR_L858R'],
}

In [45]:
threapies_lung = {
    'EGFR_L858R': 'Erlotinib',
    'EGFR_L861Q': 'Erlotinib'
}

In [46]:
lines = []
ctype = 'b'
variable_mutation_values = {'a': 1, 'b':2, 'c':3}
for v1 in var_items1: 
    for v2 in var_items2: 
        for v3 in var_items3: 
            # Sample type 
            sample_type = '_'.join([v1, v2, v3])
            # Number of mutations to be assigned to the sample
            exp_mut_count = sum([variable_mutation_values[i[1]] for i in [v1, v2, v3]])
            
            # Iterate over the 10 random samples in each possible sample type (e.g., 1a_2a_3a)
            for v4 in var_items4: 
                # Get drivers
                sample_drivers = drivers_lung_dict[v4[1]]
                # Get random passengers
                passengers_n = exp_mut_count - len(sample_drivers)
                # Add 1 passenger if none
                passengers_n = 1 if passengers_n == 0 else passengers_n
                sample_passengers = list(np.random.choice(passenger_list, size=passengers_n, replace=False))
                # Get treatments
                treatments = list(set([(driver, threapies_lung[driver]) for driver in sample_drivers if driver in threapies_lung.keys()]))
                if treatments: 
                    treatments = ':'.join(treatments[0])
                else: 
                    treatments = ''
                lines.append(pd.DataFrame([[
                    ctype,
                    sample_type, 
                    sample_type + '_' + v4,
                    exp_mut_count,
                    ';'.join(sorted(sample_drivers)), 
                    treatments,
                    ';'.join(sorted(sample_passengers)), 
                    ';'.join(sorted(sample_drivers+sample_passengers))                    
                ]]))
lung_results = pd.concat(lines)
lung_results.columns = [
    'cancer_type',
    'sample_type', 
    'sample',
    'number_mutations', 
    'drivers', 
    'treatments',
    'passengers', 
    'total'
]

In [47]:
lung_results.head(15)

,cancer_type,sample_type,sample,number_mutations,drivers,treatments,passengers,total
0,b,1a_2a_3a,1a_2a_3a_4a,3,KRAS_G12C;NFE2L2_G31R;TP53_R248G,,OR1L6_P22S,KRAS_G12C;NFE2L2_G31R;OR1L6_P22S;TP53_R248G
0,b,1a_2a_3a,1a_2a_3a_4b,3,EGFR_L858R;TP53_R273H,EGFR_L858R:Erlotinib,OR13G1_M161I,EGFR_L858R;OR13G1_M161I;TP53_R273H
0,b,1a_2a_3a,1a_2a_3a_4c,3,EGFR_L861Q;TP53_Y220C,EGFR_L861Q:Erlotinib,OR6K3_P129T,EGFR_L861Q;OR6K3_P129T;TP53_Y220C
0,b,1a_2a_3a,1a_2a_3a_4d,3,NFE2L2_G31R;TP53_R248G,,OR2M4_R234C,NFE2L2_G31R;OR2M4_R234C;TP53_R248G
0,b,1a_2a_3a,1a_2a_3a_4e,3,KEAP1_V155F;TP53_R273H,,OR56A3_G246C,KEAP1_V155F;OR56A3_G246C;TP53_R273H
0,b,1a_2a_3a,1a_2a_3a_4f,3,EGFR_L861Q,EGFR_L861Q:Erlotinib,ACTRT1_D84N;PPP1R14D_E82K,ACTRT1_D84N;EGFR_L861Q;PPP1R14D_E82K
0,b,1a_2a_3a,1a_2a_3a_4g,3,CDKN2A_P81L;TP53_L145R,,ACTRT1_D84N,ACTRT1_D84N;CDKN2A_P81L;TP53_L145R
0,b,1a_2a_3a,1a_2a_3a_4h,3,EGFR_L858R;PTEN_D162G;TP53_P151S,EGFR_L858R:Erlotinib,EIF4E1B_T218A,EGFR_L858R;EIF4E1B_T218A;PTEN_D162G;TP53_P151S
0,b,1a_2a_3a,1a_2a_3a_4i,3,EGFR_L861Q;KEAP1_C23Y,EGFR_L861Q:Erlotinib,GUCA2A_E105K,EGFR_L861Q;GUCA2A_E105K;KEAP1_C23Y
0,b,1a_2a_3a,1a_2a_3a_4j,3,EGFR_L858R,EGFR_L858R:Erlotinib,OR2T1_S267F;OR4K13_C97S,EGFR_L858R;OR2T1_S267F;OR4K13_C97S


In [48]:
# Check there are no repeated samples
len(lung_results), len(lung_results['total'].unique())

(270, 270)

In [49]:
# Export
lung_results.to_csv(lung_output_f, sep='\t', header=True, index=False)